<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/audio/sarvam_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install moviepy
! pip install librosa
! ! pip install python-dotenv
! pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
! ls drive/MyDrive/LLM

In [ ]:
import librosa
import glob

In [ ]:
audio_files = glob.glob('drive/MyDrive/LLM/*mp3')

In [ ]:
y, sr = librosa.load(audio_files[0])

In [ ]:
# Sampling rate of loaded audio -> 22050

22050

In [ ]:
import openai
from dotenv import load_dotenv
import os

_= load_dotenv('drive/MyDrive/env')
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
client = openai.OpenAI(api_key = openai.api_key)

audio_file = open(audio_files[0], "rb")
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

In [ ]:
for w in transcript.words[:5]:
  print(w['word'],w['start'],w['end'])

Congratulations 0.14000000059604645 0.6600000262260437
to 0.6600000262260437 0.9800000190734863
you 0.9800000190734863 1.1200000047683716
Mr 1.1200000047683716 1.3600000143051147
Raghavan 1.4199999570846558 1.6200000047683716


In [ ]:
def getTimeUnits(s):
  return (
      int(s/3600),
      int(s/60),
      int(s),
      int((s - int(s))*1000)
  )

In [ ]:
def displaySubs(st):
  for c in st:
    if c=='\n':
      print()
    else:
      print(c,end='')

In [ ]:
subtitleText = ''
for wcount in range(len(transcript.words[:10])):
  sh,sm,ss,sms = getTimeUnits(transcript.words[wcount]['start'])
  eh,em,es,ems = getTimeUnits(transcript.words[wcount]['end'])
  # print(str(wcount+1))
  subtitleText+=(str(wcount+1)+'\n')
  # print(sh,':',sm,':',ss,',',sms,'-->',eh,':',em,':',es,',',ems)
  subtitleText+='{}:{}:{},{} --> {}:{}:{},{}\n'.format(sh,sm,ss,sms,eh,em,es,ems)
  # print(transcript.words[wcount]['word'],'\n')
  subtitleText+=(transcript.words[wcount]['word']+'\n\n')

In [ ]:
displaySubs(subtitleText)

Forming Chunks

In [ ]:
currentChunk = []
wholeChunk = []
chunkCount = 1
chunkInterval = 15

for w in transcript.words:
  if int(w['start'])>(chunkCount*chunkInterval):
    wholeChunk.append(currentChunk)
    currentChunk = [w]
    chunkCount+=1
  elif int(w['end'])>(chunkCount*chunkInterval):
    wholeChunk.append(currentChunk)
    currentChunk = [w]
    chunkCount+=1
  else:
    currentChunk.append(w)

wholeChunk.append(currentChunk)

In [ ]:
# Making sure each word has been mapped into one chunk
s = 0
for chunk in wholeChunk:
  s+=len(chunk)

s == len(transcript.words)

True

In [ ]:
subtitleTextChunk = ''
for c in range(len(wholeChunk)):
  startTime = wholeChunk[c][0]['start']
  endTime = wholeChunk[c][-1]['end']
  sh,sm,ss,sms = getTimeUnits(startTime)
  eh,em,es,ems = getTimeUnits(endTime)
  subtitleTextChunk+=(str(c+1)+'\n')
  subtitleTextChunk+='{}:{}:{},{} --> {}:{}:{},{}\n'.format(sh,sm,ss,sms,eh,em,es,ems)
  transcription = ' '.join(map(lambda d: d['word'], wholeChunk[c]))
  subtitleTextChunk+=(transcription+'\n\n')


In [ ]:
displaySubs(subtitleTextChunk)

1
0:0:0,140 --> 0:0:15,220
Congratulations to you Mr Raghavan for that Thank you so much for joining us Over to you Hi everybody How are you I am not hearing this at all It's like a post lunch

2
0:0:15,220 --> 0:0:30,879
energy downer or something Let's hear it Are you guys awake All right You better be because we have a superstar guest here You heard the 41 million and I didn't hear honestly anything she

3
0:0:30,879 --> 0:0:45,979
said after that So we're going to ask for about 40 million from him by the end of this conversation But let's get started I want to introduce Vivek and Pratyush his co founder who's not here We

4
0:0:45,979 --> 0:1:60,819
wanted to start with playing a video of what OpenHathi does I encourage all of you to go to the website Serum ai and check it out But let me start by introducing Vivek Vivek is a dear friend and

5
0:1:60,819 --> 0:1:75,839
he's very very modest one of the most modest guys that I know But his personal journey Vivek you've been you got a